In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_excel("data/train.xlsx")
df_train = pd.DataFrame()
df_train['Date'] = pd.to_datetime(df['dt'], format="%d.%m.%Y")
df_train['Price'] = df['Цена на арматуру']
df = pd.read_excel("data/test.xlsx")
df_test = pd.DataFrame()
df_test['Date'] = pd.to_datetime(df['dt'], format="%Y.%m.%d")
df_test['Price'] = df['Цена на арматуру']
df = pd.concat((df_train, df_test)).reset_index(drop=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    428 non-null    datetime64[ns]
 1   Price   428 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.8 KB


In [6]:
df.describe()

,Date,Price
count,428,428.000000
mean,2019-02-07 12:00:00,39557.920561
min,2015-01-05 00:00:00,21000.000000
25%,2017-01-21 06:00:00,31650.000000
50%,2019-02-07 12:00:00,36150.000000
75%,2021-02-23 18:00:00,43162.500000
max,2023-03-13 00:00:00,79000.000000
std,NaN,13052.612976


Создаются X (признаки) и y (целевая переменная — Price).
Добавляются новые столбцы, такие как Price_target, Price_source и Price_Diff, которые отражают изменения цен.
Удаляются ненужные колонки (Date, Price_target).
Исключается первая строка (X.drop([0])).

In [7]:
X = df.drop(columns=['Price']).iloc[:-1, :]
y = df['Price'].values[1:]
X['Price_target'] = y
X['Price_source'] = df['Price'].values[:-1]
price_diff = X['Price_source'] - X['Price_target']
y = X['Price_target']
X = X.drop(columns=['Price_target', 'Date'])
X = X.drop([0])
X['Price_Diff'] = price_diff.values[:-1]
X

,Price_source,Price_Diff
1,32500,400
2,32000,500
3,30700,1300
4,30300,400
5,30300,0
...,...,...
422,44500,-1700
423,44500,0
424,46000,-1500
425,49200,-3200


In [9]:
from sklearn.model_selection import train_test_split
y = y[X.index]  # Пересинхронизируем индексы

X_train, X_holdout, y_train, y_holdout = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42
)

In [4]:
from lightgbm import LGBMRegressor

model0 = LGBMRegressor(n_estimators=3000, max_depth=7, learning_rate=0.03, 
                        objective='mae', metric='mae',
                        early_stopping_round=100, random_state=42)
